In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time

In [34]:
images_df=pd.read_csv('image_metadata\\images.csv')
styles_df=pd.read_csv('image_metadata\\styles.csv',on_bad_lines="skip")

In [35]:
styles_df.shape

(44424, 10)

In [36]:
images_df.shape

(44446, 2)

In [37]:
styles_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
...,...,...,...,...,...,...,...,...,...,...
44419,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013.0,Casual,Gas Men Caddy Casual Shoe
44420,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011.0,Casual,Lotto Men's Soccer Track Flip Flop
44421,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Puma Men Graphic Stellar Blue Tshirt
44422,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017.0,Casual,Rasasi Women Blue Lady Perfume


In [103]:
footwear_df=styles_df[styles_df['masterCategory']=='Footwear']

In [104]:
footwear_df['subCategory'].unique()

array(['Shoes', 'Flip Flops', 'Sandal'], dtype=object)

In [108]:
footwear_df[footwear_df['subCategory']=='Sandal']['articleType'].unique()

array(['Sandals', 'Sports Sandals', 'Flip Flops'], dtype=object)

In [109]:
sh_df=footwear_df[(footwear_df['subCategory']=='Shoes') & (footwear_df['articleType'].isin(['Casual Shoes', 'Formal Shoes','Sports Shoes']))]

ff_df=footwear_df[(footwear_df['subCategory']=='Flip Flops') & (footwear_df['articleType'].isin(['Flip Flops']))]
                                                                                                
san_df=footwear_df[(footwear_df['subCategory']=='Sandal') & (footwear_df['articleType'].isin(['Sandals']))]

footwear_df=pd.concat([sh_df, ff_df, san_df], axis=0, ignore_index=True)


In [110]:
footwear_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,9204,Men,Footwear,Shoes,Casual Shoes,Black,Summer,2011.0,Casual,Puma Men Future Cat Remix SF Black Casual Shoes
1,9036,Men,Footwear,Shoes,Formal Shoes,Black,Winter,2011.0,Formal,Buckaroo Men Flores Black Formal Shoes
2,39988,Men,Footwear,Shoes,Casual Shoes,White,Winter,2014.0,Casual,Gas Men Europa White Shoes
3,3168,Men,Footwear,Shoes,Sports Shoes,White,Fall,2010.0,Sports,Nike Men's Incinerate MSL White Blue Shoe
4,10268,Men,Footwear,Shoes,Formal Shoes,Black,Fall,2012.0,Formal,Clarks Men Hang Work Leather Black Formal Shoes
...,...,...,...,...,...,...,...,...,...,...
7321,3577,Men,Footwear,Sandal,Sandals,Black,Summer,2011.0,Casual,ADIDAS Men Black Alumin Sandal
7322,35702,Men,Footwear,Sandal,Sandals,Black,Summer,2012.0,Casual,Nike Men Solarsoft Slide Black Sandals
7323,35530,Girls,Footwear,Sandal,Sandals,Beige,Summer,2012.0,Casual,Enroute Teens Beige Sandals
7324,26543,Men,Footwear,Sandal,Sandals,Black,Fall,2012.0,Casual,ID Men Black & Yellow Sandals


In [139]:
footwear_df = (
    footwear_df
    .groupby(['subCategory', 'articleType'], group_keys=False)  # Group by subCategory and articleType
    .apply(lambda x: x.sample(min(len(x), 70)))  # Sample up to 70 rows per group
)

C:\Users\ruthu\AppData\Local\Temp\ipykernel_5836\3462751318.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  footwear_df


In [140]:
footwear_df.groupby(['subCategory','articleType'])['id'].nunique()

subCategory  articleType 
Flip Flops   Flip Flops      70
Sandal       Sandals         70
Shoes        Casual Shoes    70
             Formal Shoes    70
             Sports Shoes    70
Name: id, dtype: int64

In [159]:
clothing_df=styles_df.copy()

In [160]:
clothing_df=clothing_df[clothing_df['masterCategory']=='Apparel']

In [161]:
clothing_df['subCategory'].unique()

array(['Topwear', 'Bottomwear', 'Innerwear', 'Saree', 'Dress',
       'Loungewear and Nightwear', 'Apparel Set', 'Socks'], dtype=object)

In [162]:
clothing_df[(clothing_df['subCategory']=='Saree')]['articleType'].unique()

array(['Sarees'], dtype=object)

In [163]:
clothing_df=clothing_df[clothing_df['subCategory'].isin(['Topwear', 'Bottomwear','Saree'])]

In [164]:
clothing_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011.0,Casual,Inkfruit Mens Chain Reaction T-shirt
...,...,...,...,...,...,...,...,...,...,...
44414,30614,Men,Apparel,Topwear,Tshirts,Black,Summer,2012.0,Sports,Nike Men Striped Black Jersey
44415,13496,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Chimp Men Teja Main Hoon Blue Tshirts
44417,12544,Women,Apparel,Topwear,Tshirts,Peach,Fall,2011.0,Casual,Tantra Women Printed Peach T-shirt
44418,42234,Women,Apparel,Topwear,Tops,Blue,Summer,2012.0,Casual,Sepia Women Blue Printed Top


In [167]:
clothing_df=clothing_df[clothing_df['gender'].isin(['Women','Girls'])]

In [168]:
tw_df=clothing_df[(clothing_df['subCategory']=='Topwear') & (clothing_df['articleType'].isin(['Shirts','Tshirts','Sweaters']))]

bw_df=clothing_df[(clothing_df['subCategory']=='Bottomwear') & (clothing_df['articleType'].isin(['Jeans','Track Pants','Skirts']))]
                                                                                                
dress_df=clothing_df[(clothing_df['subCategory']=='Saree') & (clothing_df['articleType'].isin(['Sarees']))]

clothing_df=pd.concat([tw_df, bw_df, dress_df], axis=0, ignore_index=True)


In [169]:
clothing_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,26960,Women,Apparel,Topwear,Shirts,Purple,Summer,2012.0,Casual,Jealous 21 Women Purple Shirt
1,3954,Women,Apparel,Topwear,Tshirts,Pink,Summer,2011.0,Casual,Jealous 21 Women's Pink T-shirt
2,38630,Women,Apparel,Topwear,Tshirts,Purple,Summer,2012.0,Casual,Nike Women Purple Polo T-shirt
3,2288,Women,Apparel,Topwear,Tshirts,Black,Fall,2010.0,Sports,Nike Women Black T-shirt
4,9835,Women,Apparel,Topwear,Tshirts,Grey,Fall,2011.0,Casual,Little Miss Women Printed Grey T-shirt
...,...,...,...,...,...,...,...,...,...,...
2483,57976,Women,Apparel,Saree,Sarees,Pink,Summer,2012.0,Ethnic,Prafful Pink Printed Sari
2484,46007,Women,Apparel,Saree,Sarees,Blue,Summer,2012.0,Ethnic,Fabindia Blue Mull Printed Sari
2485,59644,Women,Apparel,Saree,Sarees,Cream,Fall,2012.0,Ethnic,FNF Cream & Red Wedding Collection Sari
2486,46009,Women,Apparel,Saree,Sarees,Olive,Summer,2012.0,Ethnic,Fabindia Olive Mull Hand Printed Sari


In [170]:
clothing_df.groupby(['subCategory','articleType'])['id'].nunique()

subCategory  articleType
Bottomwear   Jeans           242
             Skirts          128
             Track Pants      86
Saree        Sarees          427
Topwear      Shirts          295
             Sweaters         88
             Tshirts        1222
Name: id, dtype: int64

In [171]:
images_df['id'] = images_df['filename'].str.extract(r'(\d+)').astype(int)

In [172]:
type(images_df['id'][0])

numpy.int32

In [173]:
clothing_df['gender'].unique()

array(['Women', 'Girls'], dtype=object)

In [174]:
clothing_df = (
    clothing_df
    .groupby(['subCategory', 'articleType'], group_keys=False)  # Group by subCategory and articleType
    .apply(lambda x: x.sample(min(len(x), 70)))  # Sample up to 70 rows per group
)

C:\Users\ruthu\AppData\Local\Temp\ipykernel_5836\3313113695.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  clothing_df


In [175]:
cloth_img_df=clothing_df.merge(images_df,how='left',on="id")

In [176]:
cloth_img_df.groupby(['subCategory','articleType'])['id'].nunique()

subCategory  articleType
Bottomwear   Jeans          70
             Skirts         70
             Track Pants    70
Saree        Sarees         70
Topwear      Shirts         70
             Sweaters       70
             Tshirts        70
Name: id, dtype: int64

In [141]:
footwear_img_df=footwear_df.merge(images_df,how='left',on='id')

In [142]:
footwear_img_df.groupby(['subCategory','articleType'])['id'].nunique()

subCategory  articleType 
Flip Flops   Flip Flops      70
Sandal       Sandals         70
Shoes        Casual Shoes    70
             Formal Shoes    70
             Sports Shoes    70
Name: id, dtype: int64

In [143]:
footwear_img_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename,link
0,15738,Men,Footwear,Flip Flops,Flip Flops,Black,Summer,2012.0,Casual,Nike Men Black Flip Flops,15738.jpg,http://assets.myntassets.com/v1/images/style/p...
1,17651,Unisex,Footwear,Flip Flops,Flip Flops,Blue,Spring,2013.0,Casual,Converse Unisex Blue Flip Flops,17651.jpg,http://assets.myntassets.com/v1/images/style/p...
2,44209,Women,Footwear,Flip Flops,Flip Flops,Silver,Winter,2012.0,Casual,iPanema Women Silver Flip Flops,44209.jpg,http://assets.myntassets.com/v1/images/style/p...
3,15387,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2012.0,Casual,Fila Men Kumi Red Slippers,15387.jpg,http://assets.myntassets.com/v1/images/style/p...
4,21202,Men,Footwear,Flip Flops,Flip Flops,Multi,Summer,2012.0,Casual,Quiksilver Men Multi Milokai Print Flip Flops,21202.jpg,http://assets.myntassets.com/v1/images/style/p...
...,...,...,...,...,...,...,...,...,...,...,...,...
345,1786,Men,Footwear,Shoes,Sports Shoes,White,Fall,2010.0,Sports,Lotto Men's Court Logo White Shoe,1786.jpg,http://assets.myntassets.com/v1/images/style/p...
346,15719,Men,Footwear,Shoes,Sports Shoes,Fluorescent Green,Fall,2011.0,Sports,Nike Men Egoli FG Fluorescent Green Sports Shoes,15719.jpg,http://assets.myntassets.com/v1/images/style/p...
347,23929,Men,Footwear,Shoes,Sports Shoes,Silver,Fall,2011.0,Sports,Lotto Men Silver Detroit Sports Shoes,23929.jpg,http://assets.myntassets.com/v1/images/style/p...
348,5393,Men,Footwear,Shoes,Sports Shoes,White,Summer,2011.0,Sports,Kipsta Match Up V2 Black White,5393.jpg,http://assets.myntassets.com/v1/images/style/p...


In [177]:
cloth_img_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename,link
0,24607,Women,Apparel,Bottomwear,Jeans,Blue,Fall,2011.0,Casual,Spykar Women Blue Washed Jeans,24607.jpg,http://assets.myntassets.com/v1/images/style/p...
1,13259,Women,Apparel,Bottomwear,Jeans,Black,Fall,2011.0,Casual,Palm Tree Kids Girl Washed Black Jeans,13259.jpg,http://assets.myntassets.com/v1/images/style/p...
2,7188,Women,Apparel,Bottomwear,Jeans,Blue,Fall,2011.0,Casual,Jealous 21 Women Supper Skinny Zipped Medium B...,7188.jpg,http://assets.myntassets.com/v1/images/style/p...
3,7709,Women,Apparel,Bottomwear,Jeans,Black,Fall,2011.0,Casual,Jealous 21 Women's Aaren Black Jegging,7709.jpg,http://assets.myntassets.com/v1/images/style/p...
4,50942,Women,Apparel,Bottomwear,Jeans,Blue,Summer,2013.0,Casual,Lee Women Blue Jeans,50942.jpg,http://assets.myntassets.com/v1/images/style/p...
...,...,...,...,...,...,...,...,...,...,...,...,...
485,3893,Girls,Apparel,Topwear,Tshirts,Green,Summer,2011.0,Casual,Disney Kids Girl's Green Minnie Top With Pink ...,3893.jpg,http://assets.myntassets.com/v1/images/style/p...
486,6233,Women,Apparel,Topwear,Tshirts,Green,Summer,2011.0,Casual,UCB Women's Basic Neck Green T-shirt,6233.jpg,http://assets.myntassets.com/v1/images/style/p...
487,3071,Women,Apparel,Topwear,Tshirts,Blue,Summer,2011.0,Casual,Tantra Women's Shlokas Blue T-shirt,3071.jpg,http://assets.myntassets.com/v1/images/style/p...
488,54964,Women,Apparel,Topwear,Tshirts,Cream,Summer,2012.0,Casual,Do U Speak Green Women Cream T-Shirt,54964.jpg,http://assets.myntassets.com/v1/images/style/p...


In [192]:
output_dir=os.path.join(".","images","clothing")


In [193]:
cloth_img_df.columns

Index(['id', 'gender', 'masterCategory', 'subCategory', 'articleType',
       'baseColour', 'season', 'year', 'usage', 'productDisplayName',
       'filename', 'link'],
      dtype='object')

In [194]:
final_cloth_df=pd.DataFrame(columns=['id','masterCategory', 'subCategory', 'articleType','filename'])

In [195]:
for index, row in cloth_img_df.iterrows():
    
    link = row['link']
    file_name=row['filename']
    #print(f"Processing {filename} from {link}")
    
    try:
        # Example: Download and save the file
        response = requests.get(link, stream=True)
        if response.status_code == 200:
            with open(os.path.join(output_dir, file_name), 'wb') as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            #print(f"Downloaded: {filename}")
            new_row = {
                'id': row['id'],
                'masterCategory': row['masterCategory'],
                'subCategory': row['subCategory'],
                'articleType': row['articleType'],
                'filename': row['filename']
            }
            final_cloth_df = pd.concat([final_cloth_df, pd.DataFrame([new_row])], ignore_index=True)
        else:
            print(f"Failed to download {file_name} ")

    except requests.exceptions.RequestException as e:
        print(f"exception-Failed to download {file_name}")

In [196]:
final_cloth_df

,id,masterCategory,subCategory,articleType,filename
0,24607,Apparel,Bottomwear,Jeans,24607.jpg
1,13259,Apparel,Bottomwear,Jeans,13259.jpg
2,7188,Apparel,Bottomwear,Jeans,7188.jpg
3,7709,Apparel,Bottomwear,Jeans,7709.jpg
4,50942,Apparel,Bottomwear,Jeans,50942.jpg
...,...,...,...,...,...
485,3893,Apparel,Topwear,Tshirts,3893.jpg
486,6233,Apparel,Topwear,Tshirts,6233.jpg
487,3071,Apparel,Topwear,Tshirts,3071.jpg
488,54964,Apparel,Topwear,Tshirts,54964.jpg


In [197]:
output_dir=os.path.join(".","images","foot wear")
final_footwear_df=pd.DataFrame(columns=['id','masterCategory', 'subCategory', 'articleType','filename'])

In [198]:
footwear_img_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename,link
0,15738,Men,Footwear,Flip Flops,Flip Flops,Black,Summer,2012.0,Casual,Nike Men Black Flip Flops,15738.jpg,http://assets.myntassets.com/v1/images/style/p...
1,17651,Unisex,Footwear,Flip Flops,Flip Flops,Blue,Spring,2013.0,Casual,Converse Unisex Blue Flip Flops,17651.jpg,http://assets.myntassets.com/v1/images/style/p...
2,44209,Women,Footwear,Flip Flops,Flip Flops,Silver,Winter,2012.0,Casual,iPanema Women Silver Flip Flops,44209.jpg,http://assets.myntassets.com/v1/images/style/p...
3,15387,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2012.0,Casual,Fila Men Kumi Red Slippers,15387.jpg,http://assets.myntassets.com/v1/images/style/p...
4,21202,Men,Footwear,Flip Flops,Flip Flops,Multi,Summer,2012.0,Casual,Quiksilver Men Multi Milokai Print Flip Flops,21202.jpg,http://assets.myntassets.com/v1/images/style/p...
...,...,...,...,...,...,...,...,...,...,...,...,...
345,1786,Men,Footwear,Shoes,Sports Shoes,White,Fall,2010.0,Sports,Lotto Men's Court Logo White Shoe,1786.jpg,http://assets.myntassets.com/v1/images/style/p...
346,15719,Men,Footwear,Shoes,Sports Shoes,Fluorescent Green,Fall,2011.0,Sports,Nike Men Egoli FG Fluorescent Green Sports Shoes,15719.jpg,http://assets.myntassets.com/v1/images/style/p...
347,23929,Men,Footwear,Shoes,Sports Shoes,Silver,Fall,2011.0,Sports,Lotto Men Silver Detroit Sports Shoes,23929.jpg,http://assets.myntassets.com/v1/images/style/p...
348,5393,Men,Footwear,Shoes,Sports Shoes,White,Summer,2011.0,Sports,Kipsta Match Up V2 Black White,5393.jpg,http://assets.myntassets.com/v1/images/style/p...


In [199]:
for index, row in footwear_img_df.iterrows():
    
    link = row['link']
    file_name=row['filename']
    #print(f"Processing {filename} from {link}")
    
    try:
        # Example: Download and save the file
        response = requests.get(link, stream=True)
        if response.status_code == 200:
            with open(os.path.join(output_dir, file_name), 'wb') as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            #print(f"Downloaded: {filename}")
            new_row = {
                'id': row['id'],
                'masterCategory': row['masterCategory'],
                'subCategory': row['subCategory'],
                'articleType': row['articleType'],
                'filename': row['filename']
            }
            final_footwear_df = pd.concat([final_footwear_df, pd.DataFrame([new_row])], ignore_index=True)
        else:
            print(f"Failed to download {file_name} ")

    except requests.exceptions.RequestException as e:
        print(f"exception-Failed to download {file_name}")

Failed to download 33339.jpg 
Failed to download 34727.jpg 
Failed to download 17982.jpg 


In [201]:
final_footwear_df.groupby(['subCategory','articleType'])['id'].nunique()

subCategory  articleType 
Flip Flops   Flip Flops      70
Sandal       Sandals         70
Shoes        Casual Shoes    70
             Formal Shoes    69
             Sports Shoes    68
Name: id, dtype: int64

In [202]:
final_footwear_df

,id,masterCategory,subCategory,articleType,filename
0,15738,Footwear,Flip Flops,Flip Flops,15738.jpg
1,17651,Footwear,Flip Flops,Flip Flops,17651.jpg
2,44209,Footwear,Flip Flops,Flip Flops,44209.jpg
3,15387,Footwear,Flip Flops,Flip Flops,15387.jpg
4,21202,Footwear,Flip Flops,Flip Flops,21202.jpg
...,...,...,...,...,...
342,1786,Footwear,Shoes,Sports Shoes,1786.jpg
343,15719,Footwear,Shoes,Sports Shoes,15719.jpg
344,23929,Footwear,Shoes,Sports Shoes,23929.jpg
345,5393,Footwear,Shoes,Sports Shoes,5393.jpg


In [206]:
final_cloth_df.to_csv('final_cloth_img.csv',index=False)
final_footwear_df.to_csv('final_footwear_img.csv',index=False)